
# AG-3 Actividad Guiada 
## Nombre: Eduardo Cárdenas 
###URL: https://colab.research.google.com/drive/1pFQ27d7MA4TjnH5bosEEtkvXcqXHSUdE?usp=sharing


In [ ]:
!pip install request
!pip install tsplib95

In [ ]:
import urllib.request

In [ ]:
file = 'swiss42'
urllib.request.urlretrieve('http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp',file)

In [ ]:
import tsplib95
import random
from math import e ,inf, exp

In [ ]:
problem = tsplib95.load(file)

Nodos = list(problem.get_nodes())
Aristas = list(problem.get_edges())

$$\frac{(n-1)!}{2} = \frac {334525...........}{2}$$

In [ ]:
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion

crear_solucion(Nodos)

def distancia(a,b, problem):
  return  problem.get_weight(a,b)

distancia(0,1,problem)

def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

s= crear_solucion(Nodos)
distancia_total(s, problem)

In [ ]:
def busqueda_aleatoria(problem, N):

  mejor_solucion = []
  mejor_distancia= 10e10
  
  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion, problem)

    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia

  print("Mejor solución: ", mejor_solucion)
  print("Distancia    : ", mejor_distancia)
  return mejor_solucion

sol = busqueda_aleatoria(problem, 10000)
#print(distancia_total(s, problem), s)

In [ ]:
#Busqueda Local

def genera_vecina(solucion):
  #Generador de soluciones vecinas
  mejor_solucion = []
  mejor_distancia = 10e100

  for i in range(1, len(solucion)-1):
    for j in range(i+1, len(solucion)):

      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina =  distancia_total(vecina, problem)

      if distancia_vecina < mejor_distancia:
        mejor_solucion = vecina
        mejor_distancia = distancia_vecina
    
  return mejor_solucion

s1 = crear_solucion(Nodos)

s2 = genera_vecina(s1)

print(distancia_total(s1, problem), s1)
print(distancia_total(s2, problem), s2)

In [ ]:
def busqueda_local(problem):
  mejor_solucion = []

  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = 10e100

  iteracion = 0 
  while(1):
    iteracion =+ 1
    vecina = genera_vecina(solucion_referencia)
    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
    else:
      print("En la iteracion ", iteracion, " encontramos la solucion:", mejor_solucion )
      print("Distancia: ", mejor_distancia)
      return mejor_solucion
    solucion_referencia = vecina

In [ ]:
sol = busqueda_local(problem)

In [ ]:
def genera_vecina_aleatoria(solucion):
    i,j = sorted(random.sample( range(1,len(solucion)), 2))
    return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
print(genera_vecina_aleatoria(s1))

In [ ]:
def bajar_temperatura(T):
  return T*0.999
bajar_temperatura(37)

In [ ]:
def probabilidad(T,d):
  if random.random() <= exp(-1*d / T):
    return True
  else: 
    return False
probabilidad(1000, 100)

#Recocido Simulado

In [ ]:
def recocido_simulado(problem,TEMPERATURA=1000):
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)

  mejor_solucion = []
  mejor_distancia = 10e100

  N=0
  while TEMPERATURA> .0001:
    N += 1
    vecina = genera_vecina_aleatoria(solucion_referencia)

    distancia_vecina = distancia_total(vecina, problem)

    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina

    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ):
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina

    TEMPERATURA = bajar_temperatura(TEMPERATURA)

  print("La mejor solucion encontrada es ", end="")
  print(mejor_solucion)
  print("con una distancia total de ", end="")
  print(mejor_distancia)
  return mejor_solucion

sol = recocido_simulado(problem, 1000000)


#Colonia de Homrigas

In [ ]:
def Add_Nodo(problem, H, T):
  Nodos = list(problem.get_nodes())
  return random.choice( list(set(range(1,len(Nodos))) - set(H) ) )
  

In [ ]:
def Incrementa_Feromona(problem, T, H):
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

In [ ]:
def Evaporar_Feromonas(T ):
  T= [[max(T[i][j] - 0.3, 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [ ]:
def hormigas(problem, N) :

  Nodos = list(problem.get_nodes())

  Aristas = list(problem.get_edges())

  T = [[1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  Hormiga = [[0] for _ in range(N)]

  for h in range(N) :
    for i in range(len(Nodos)-1):

      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h],T )
      Hormiga[h].append(Nuevo_Nodo)

    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    T = Evaporar_Feromonas(T)
  mejor_solucion = []
  mejor_distancia = 10e100

  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual  < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia = distancia_actual

  print(mejor_solucion)
  print("La mejor distancias es: ", mejor_distancia)
  return(mejor_solucion,mejor_distancia)


In [ ]:
sol = hormigas(problem, 500)